In [1]:
import pandas as pd
import os

In [2]:
df = pd.read_parquet('../data/yellow_tripdata_2023-01.parquet')
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


## Q1. Downloading the data

We'll use [the same NYC taxi dataset](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page),
but instead of "**Green** Taxi Trip Records", we'll use "**Yellow** Taxi Trip Records".

Download the data for January and February 2023.

Read the data for January. How many columns are there?

* 16
* 17
* 18
* 19

In [3]:
df.shape

(3066766, 19)

## Q2. Computing duration

Now let's compute the `duration` variable. It should contain the duration of a ride in minutes. 

What's the standard deviation of the trips duration in January?

* 32.59
* 42.59
* 52.59
* 62.59

In [4]:
df.sample()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
2638200,1,2023-01-28 07:54:59,2023-01-28 08:04:33,0.0,1.4,1.0,N,229,114,1,10.0,2.5,0.5,2.8,0.0,1.0,16.8,2.5,0.0


In [5]:
df['duration'] = (df.tpep_dropoff_datetime - df.tpep_pickup_datetime).dt.total_seconds() / 60
df['duration'].sample()

746584    17.0
Name: duration, dtype: float64

In [6]:
describe_df = df['duration'].describe()
describe_df = describe_df.apply(lambda x: f'{x:.2f}')
describe_df

count    3066766.00
mean          15.67
std           42.59
min          -29.20
25%            7.12
50%           11.52
75%           18.30
max        10029.18
Name: duration, dtype: object

## Q3. Dropping outliers

Next, we need to check the distribution of the `duration` variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

* 90%
* 92%
* 95%
* 98%

In [7]:
df_filter = df[(df['duration'] >= 1) & (df['duration'] <= 60)]
print(f'The fraction left after the outlier removal: {round(df_filter.shape[0] / df.shape[0] * 100,2)}%')

The fraction left after the outlier removal: 98.12%


## Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model. 

* Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will 
  label encode them)
* Fit a dictionary vectorizer 
* Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?

* 2
* 155
* 345
* 515
* 715

In [8]:
from sklearn.feature_extraction import DictVectorizer

In [9]:
# Nos aseguramos de que los IDs sean de tipo string
df_filter['PULocationID'] = df_filter['PULocationID'].astype(str)
df_filter['DOLocationID'] = df_filter['DOLocationID'].astype(str)

# Convertimos el dataframe a una lista de diccionarios, usando solo las dos columnas necesarias
dict_list = df_filter[['PULocationID', 'DOLocationID']].to_dict(orient='records')

/var/folders/7b/x372r2jn4b95xj0_xs49ys4h0000gn/T/ipykernel_62137/2125201322.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filter['PULocationID'] = df_filter['PULocationID'].astype(str)
/var/folders/7b/x372r2jn4b95xj0_xs49ys4h0000gn/T/ipykernel_62137/2125201322.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filter['DOLocationID'] = df_filter['DOLocationID'].astype(str)


In [10]:
# Creamos una instancia de DictVectorizer
dv = DictVectorizer(sparse=False)

# Ajustamos y transformamos la lista de diccionarios para obtener la matriz de características
feature_matrix = dv.fit_transform(dict_list)


In [11]:
# Obtenemos los nombres de las características
feature_names = dv.get_feature_names_out()


In [12]:
len(feature_names)

515

In [13]:
print(pd.DataFrame(feature_matrix, columns=feature_names).shape)

(3009173, 515)


## Q5. Training a model

Now let's use the feature matrix from the previous step to train a model. 

* Train a plain linear regression model with default parameters 
* Calculate the RMSE of the model on the training data

What's the RMSE on train?

* 3.64
* 7.64
* 11.64
* 16.64

In [14]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [15]:
X = feature_matrix
y = df_filter['duration'].values

In [16]:
model = LinearRegression()
model.fit(X, y) 
y_pred = model.predict(X)
print(f'RMSE: {np.sqrt(mean_squared_error(y, y_pred))}')

In [ ]:
import pickle
# Guardamos el modelo para no tener que reentrenarlo

with open('linear_regresion_model.pkl', 'wb') as f:
    pickle.dump((dv, model), f)

In [ ]:
# abrimos el pkl con el modelo cargado
with open('linear_regresion_model.pkl', 'rb') as f:
    dv, model = pickle.load(f)

## Q6. Evaluating the model

Now let's apply this model to the validation dataset (February 2023). 

What's the RMSE on validation?

* 3.81
* 7.81
* 11.81
* 16.81

In [ ]:
feb_data = pd.read_parquet('../data/yellow_tripdata_2022-02.parquet')